# Create and manage a batch endpoint for inferencing

**Motivations** - In this example, we're going to deploy a model to solve the classic MNIST ("Modified National Institute of Standards and Technology") digit recognition problem to perform batch inferencing over large amounts of data (image files). In the first section of this tutorial, we're going to create a batch deployment with a model created using Torch. Such deployment will become our default one in the endpoint. In the second half, we're going to see how we can create a second deployment using a model created with TensorFlow (Keras), test it out, and then switch the endpoint to start using the new deployment as default once we confirm it is working.. 

# 1. Connect to Azure Machine Learning Workspace

The [workspace](https://docs.microsoft.com/en-us/azure/machine-learning/concept-workspace) is the top-level resource for Azure Machine Learning, providing a centralized place to work with all the artifacts you create when you use Azure Machine Learning. In this section we will connect to the workspace in which the job will be run.

## 1.1. Import the required libraries

In [ ]:
from azure.ai.ml import MLClient, Input
from azure.ai.ml.entities import (
    BatchEndpoint,
    ModelBatchDeployment,
    ModelBatchDeploymentSettings,
    Model,
    Environment,
    AmlCompute,
    BatchRetrySettings,
    CodeConfiguration,
)
from azure.identity import DefaultAzureCredential
from azure.ai.ml.constants import AssetTypes, BatchDeploymentOutputAction

## 1.2. Configure workspace details and get a handle to the workspace

To connect to a workspace, we need identifier parameters - a subscription, resource group and workspace name. We will use these details in the `MLClient` from `azure.ai.ml` to get a handle to the required Azure Machine Learning workspace. We use the default [default azure authentication](https://docs.microsoft.com/en-us/python/api/azure-identity/azure.identity.defaultazurecredential?view=azure-python) for this tutorial. Check the [configuration notebook](../../jobs/configuration.ipynb) for more details on how to configure credentials and connect to a workspace.

In [ ]:
subscription_id = "<SUBSCRIPTION_ID>"
resource_group = "<RESOURCE_GROUP>"
workspace = "<AML_WORKSPACE_NAME>"

In [ ]:
ml_client = MLClient(
    DefaultAzureCredential(), subscription_id, resource_group, workspace
)

If you are working in a Azure Machine Learning compute, you can simply:

In [ ]:
ml_client = MLClient.from_config(DefaultAzureCredential())

# 2. Create Batch Endpoint
Batch endpoints are endpoints that are used batch inferencing on large volumes of data over a period of time. Batch endpoints receive pointers to data and run jobs asynchronously to process the data in parallel on compute clusters. Batch endpoints store outputs to a data store for further analysis.

To create an online endpoint we will use `BatchEndpoint`. This class allows user to configure the following key aspects:
- `name` - Name of the endpoint. Needs to be unique at the Azure region level
- `auth_mode` - The authentication method for the endpoint. Currently only Azure Active Directory (Azure AD) token-based (`aad_token`) authentication is supported. 
- `description`- Description of the endpoint.

## 2.1 Configure the endpoint

In [ ]:
endpoint_name = "mnist-batch"

In [ ]:
import random
import string

# Creating a unique endpoint name by including a random suffix
allowed_chars = string.ascii_lowercase + string.digits
endpoint_suffix = "".join(random.choice(allowed_chars) for x in range(5))
endpoint_name = f"{endpoint_name}-{endpoint_suffix}"

print(f"Endpoint name: {endpoint_name}")

Let's configure the endpoint:

In [ ]:
endpoint = BatchEndpoint(
    name=endpoint_name,
    description="A batch endpoint for scoring images from the MNIST dataset.",
    tags={"type": "deep-learning"},
)

## 2.2 Create the endpoint
Using the `MLClient` created earlier, we will now create the Endpoint in the workspace. This command will start the endpoint creation and return a confirmation response while the endpoint creation continues.

In [ ]:
ml_client.begin_create_or_update(endpoint).result()

## 3. Registering the model

### 3.1 About the model

We are going to deploy a model created using Torch to solve the typical MNIST classification problem.

### 3.2 Registering the model in the workspace

We need to register the model in order to use it with Azure Machine Learning:

In [ ]:
model_name = "mnist-classifier-torch"
model_local_path = "deployment-torch/model/"

model = ml_client.models.create_or_update(
    Model(
        name=model_name,
        path=model_local_path,
        type=AssetTypes.CUSTOM_MODEL,
        tags={"task": "classification", "framework": "torch"},
    )
)

Let's get a reference to the model:

In [ ]:
model = ml_client.models.get(name=model_name, label="latest")

# 4. Create a deployment
A deployment is a set of resources required for hosting the model that does the actual inferencing.

### 4.1 Creating an scoring script to work with the model

In [ ]:
%%writefile deployment-torch/code/batch_driver.py

import os
import pandas as pd
import torch
import torchvision
import glob
from os.path import basename
from mnist_classifier import MnistClassifier
from typing import List

def init():
    global model
    global device

    # AZUREML_MODEL_DIR is an environment variable created during deployment
    # It is the path to the model folder
    model_path = os.environ["AZUREML_MODEL_DIR"]
    model_file = glob.glob(f"{model_path}/*/*.pt")[-1]

    model = MnistClassifier()
    model.load_state_dict(torch.load(model_file))
    model.eval()

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def run(mini_batch: List[str]) -> pd.DataFrame:
    print(f"Executing run method over batch of {len(mini_batch)} files.")

    results = []
    with torch.no_grad():
        for image_path in mini_batch:
            image_data = torchvision.io.read_image(image_path).float()
            batch_data = image_data.expand(1, -1, -1, -1)
            input = batch_data.to(device)

            # perform inference
            predict_logits = model(input)

            # Compute probabilities, classes and labels
            predictions = torch.nn.Softmax(dim=-1)(predict_logits)
            predicted_prob, predicted_class= torch.max(predictions, axis=-1)
            
            results.append({ 
                "file": basename(image_path),
                "class": predicted_class.numpy()[0],
                "probability": predicted_prob.numpy()[0]
            })

    return pd.DataFrame(results)

### 4.2 Creating the compute

Batch deployments can run on any Azure ML compute that already exists in the workspace. That means that multiple batch deployments can share the same compute infrastructure. In this example, we are going to work on an AzureML compute cluster called `cpu-cluster`. Let's verify the compute exists on the workspace or create it otherwise.

In [ ]:
compute_name = "batch-cluster"
if not any(filter(lambda m: m.name == compute_name, ml_client.compute.list())):
    compute_cluster = AmlCompute(
        name=compute_name,
        description="CPU cluster compute",
        min_instances=0,
        max_instances=2,
    )
    ml_client.compute.begin_create_or_update(compute_cluster).result()

Compute may take time to be created. Let's wait for it:

In [ ]:
from time import sleep

print(f"Waiting for compute {compute_name}", end="")
while ml_client.compute.get(name=compute_name).provisioning_state == "Creating":
    sleep(1)
    print(".", end="")

print(" [DONE]")

### 4.3 Creating the environment

Let's create the environment. In our case, our model runs on `Torch`. Azure Machine Learning already has an environment with the required software installed, so we can reutilize this environment.

In [ ]:
env = Environment(
    name="batch-torch-py38",
    conda_file="deployment-torch/environment/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu22.04:latest",
)

### 4.3 Configure the deployment

We will create a deployment for our endpoint using the `BatchDeployment` class. This class allows user to configure the following key aspects.
- `name` - Name of the deployment.
- `endpoint_name` - Name of the endpoint to create the deployment under.
- `model` - The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.
- `environment` - The environment to use for the deployment. This value can be either a reference to an existing versioned environment in the workspace or an inline environment specification.
- `code_path`- Path to the source code directory for scoring the model
- `scoring_script` - Relative path to the scoring file in the source code directory
- `compute` - Name of the compute target to execute the batch scoring jobs on
- `instance_count`- The number of nodes to use for each batch scoring job.		1
- `max_concurrency_per_instance`- The maximum number of parallel scoring_script runs per instance.
- `mini_batch_size`	- The number of files the code_configuration.scoring_script can process in one `run()` call.
- `retry_settings`- Retry settings for scoring each mini batch.		
   - `max_retries`- The maximum number of retries for a failed or timed-out mini batch (default is 3)
   - `timeout`- The timeout in seconds for scoring a mini batch (default is 30)
- `output_action`- Indicates how the output should be organized in the output file. Allowed values are `append_row` or `summary_only`. Default is `append_row`
- `output_file_name`- Name of the batch scoring output file. Default is `predictions.csv`
- `environment_variables`- Dictionary of environment variable name-value pairs to set for each batch scoring job.
- `logging_level`- The log verbosity level.	Allowed values are `warning`, `info`, `debug`. Default is `info`.

In [ ]:
deployment = ModelBatchDeployment(
    name="mnist-torch-dpl",
    description="A deployment using Torch to solve the MNIST classification dataset.",
    endpoint_name=endpoint_name,
    model=model,
    code_configuration=CodeConfiguration(
        code="deployment-torch/code/", scoring_script="batch_driver.py"
    ),
    environment=env,
    compute=compute_name,
    settings=ModelBatchDeploymentSettings(
        max_concurrency_per_instance=2,
        mini_batch_size=10,
        instance_count=2,
        output_action=BatchDeploymentOutputAction.APPEND_ROW,
        output_file_name="predictions.csv",
        retry_settings=BatchRetrySettings(max_retries=3, timeout=30),
        logging_level="info",
    ),
)

### 4.4 Create the deployment
Using the `MLClient` created earlier, we will now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [ ]:
ml_client.begin_create_or_update(deployment).result()

Let's update the default deployment name in the endpoint:

In [ ]:
endpoint = ml_client.batch_endpoints.get(endpoint_name)
endpoint.defaults.deployment_name = deployment.name
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

We can see the details of the deployment as follows:

In [ ]:
ml_client.batch_deployments.get(name=deployment.name, endpoint_name=endpoint.name)

### 4.5 Test the endpoint with sample data
Using the `MLClient` created earlier, we will get a handle to the endpoint. The endpoint can be invoked using the `invoke` command with the following parameters:
- `name` - Name of the endpoint
- `input_path` - Path where input data is present
- `deployment_name` - Name of the specific deployment to test in an endpoint

In [ ]:
import time

# Let's sleep for 2 min to ensure all resources are ready. This is only for automation purposes.
time.sleep(120)

Using Zipfile to extract data into a temporary folder before running the scoring jobs

In [ ]:
from zipfile import ZipFile
from pathlib import Path

repo_root = Path.cwd().resolve().parents[5]

dataset_zipfile_path = repo_root / "assets/batch-endpoints-data/mnist-batch/data.zip"
print(dataset_zipfile_path)

temp_data_path = "."
with ZipFile(dataset_zipfile_path, "r") as zip_ref:
    zip_ref.extractall(temp_data_path)

#### 4.5.1 Invoke the endpoint

Let's now invoke the endpoint for batch scoring job:

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name=deployment.name,
    input=Input(
        path=Path(temp_data_path) / "data",
        type=AssetTypes.URI_FOLDER,
    ),
)

#### 4.5.2 Get the details of the invoked job
Let us get details and logs of the invoked job

In [ ]:
ml_client.jobs.get(job.name)

We can wait for the job to finish using the following code:

In [ ]:
ml_client.jobs.stream(job.name)

#### 4.5.3 Download the results

The deployment creates a child job that executes the scoring. We can get the details of it using the following code:

In [ ]:
scoring_job = list(ml_client.jobs.list(parent_job_name=job.name))[0]

In [ ]:
print("Job name:", scoring_job.name)
print("Job status:", scoring_job.status)
print(
    "Job duration:",
    scoring_job.creation_context.last_modified_at
    - scoring_job.creation_context.created_at,
)

The outputs generated by the deployment job will be placed in an output named `score`:

In [ ]:
ml_client.jobs.download(name=scoring_job.name, download_path=".", output_name="score")

We can read this data using pandas library:

In [ ]:
import pandas as pd

score = pd.read_csv(
    "named-outputs/score/predictions.csv",
    header=None,
    names=["file", "class"],
    sep=" ",
)
score

### 4.6 Override deployment configuration at invocation time

#### 4.6.1 Output location

You can indicate the output path where you want the job to place the results. To do that, let's first find the ID of a data store registered in AzureML. You can only place outputs in data stores:

In [ ]:
batch_ds = ml_client.datastores.get_default()

Run the job:

In [ ]:
filename = f"predictions-{random.randint(0,99999)}.csv"

job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    input=Input(
        path=Path(temp_data_path) / "data",
        type=AssetTypes.URI_FOLDER,
    ),
    params_override=[
        {"output_dataset.datastore_id": f"azureml:{batch_ds.id}"},
        {"output_dataset.path": f"/{endpoint_name}/"},
        {"output_file_name": filename},
    ],
)

#### 4.6.1 Override deployment configuration settings

Some other parameters can be override, including `mini_batch_size` and `instance_count`:

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    input=Input(path="./data"),
    params_override=[{"mini_batch_size": "20"}, {"compute.instance_count": "5"}],
)

# 5. Create another deployment

Let's now create a second deployment, but this time we will solve the same problem using a model trained with TensorFlow and Keras.

In [ ]:
model_name = "mnist-classifier-keras"
model_local_path = "deployment-keras/model/"

model = ml_client.models.create_or_update(
    Model(
        name=model_name,
        path=model_local_path,
        type=AssetTypes.CUSTOM_MODEL,
        tags={"task": "classification", "framework": "tensorflow"},
    )
)

Let's get a reference to the model:

In [ ]:
model = ml_client.models.get(name=model_name, label="latest")

### 5.1 Creating an scoring script to work with the model

In [ ]:
%%writefile deployment-keras/code/batch_driver.py

import os
import numpy as np
import pandas as pd
import tensorflow as tf
from typing import List
from os.path import basename
from PIL import Image
from tensorflow.keras.models import load_model


def init():
    global model

    # AZUREML_MODEL_DIR is an environment variable created during deployment
    model_path = os.path.join(os.environ["AZUREML_MODEL_DIR"], "model")

    # load the model
    model = load_model(model_path)

def run(mini_batch : List[str]) -> pd.DataFrame:
    print(f"Executing run method over batch of {len(mini_batch)} files.")

    results = []
    for image_path in mini_batch:
        data = Image.open(image_path)
        data = np.array(data)
        data_batch = tf.expand_dims(data, axis=0)

        # perform inference
        pred = model.predict(data_batch)

        # Compute probabilities, classes and labels
        pred_prob = tf.math.reduce_max(tf.math.softmax(pred, axis=-1)).numpy()
        pred_class = tf.math.argmax(pred, axis=-1).numpy()

        results.append({
            "file": basename(image_path), 
            "class": pred_class[0],
            "probability": pred_prob
        })

    return pd.DataFrame(results)


### 5.2 Creating the compute

Batch deployments can run on any Azure ML compute that already exists in the workspace. That means that multiple batch deployments can share the same compute infrastructure. In this example, we are going to work on an AzureML compute cluster called `cpu-cluster`.

### 5.3 Creating the environment

Let's create the environment. In our case, our model runs on `Torch`. Azure Machine Learning already has an environment with the required software installed, so we can reutilize this environment.

In [ ]:
env = Environment(
    name="batch-tensorflow-py38",
    conda_file="deployment-keras/environment/conda.yaml",
    image="mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu22.04:latest",
)

### 5.4 Configure the deployment

We will create a deployment for our endpoint using the `BatchDeployment` class. This class allows user to configure the following key aspects.
- `name` - Name of the deployment.
- `endpoint_name` - Name of the endpoint to create the deployment under.
- `model` - The model to use for the deployment. This value can be either a reference to an existing versioned model in the workspace or an inline model specification.
- `environment` - The environment to use for the deployment. This value can be either a reference to an existing versioned environment in the workspace or an inline environment specification.
- `code_path`- Path to the source code directory for scoring the model
- `scoring_script` - Relative path to the scoring file in the source code directory
- `compute` - Name of the compute target to execute the batch scoring jobs on
- `instance_count`- The number of nodes to use for each batch scoring job.		1
- `max_concurrency_per_instance`- The maximum number of parallel scoring_script runs per instance.
- `mini_batch_size`	- The number of files the code_configuration.scoring_script can process in one `run()` call.
- `retry_settings`- Retry settings for scoring each mini batch.		
   - `max_retries`- The maximum number of retries for a failed or timed-out mini batch (default is 3)
   - `timeout`- The timeout in seconds for scoring a mini batch (default is 30)
- `output_action`- Indicates how the output should be organized in the output file. Allowed values are `append_row` or `summary_only`. Default is `append_row`
- `output_file_name`- Name of the batch scoring output file. Default is `predictions.csv`
- `environment_variables`- Dictionary of environment variable name-value pairs to set for each batch scoring job.
- `logging_level`- The log verbosity level.	Allowed values are `warning`, `info`, `debug`. Default is `info`.

In [ ]:
deployment_keras = ModelBatchDeployment(
    name="mnist-keras-dpl",
    description="A deployment using Keras to solve the MNIST classification dataset.",
    endpoint_name=endpoint_name,
    model=model,
    code_configuration=CodeConfiguration(
        code="deployment-keras/code/", scoring_script="batch_driver.py"
    ),
    environment=env,
    compute=compute_name,
    settings=ModelBatchDeploymentSettings(
        instance_count=2,
        max_concurrency_per_instance=2,
        mini_batch_size=10,
        output_action=BatchDeploymentOutputAction.APPEND_ROW,
        output_file_name="predictions.csv",
        retry_settings=BatchRetrySettings(max_retries=3, timeout=30),
        logging_level="info",
    ),
)

### 5.5 Create the deployment
Using the `MLClient` created earlier, we will now create the deployment in the workspace. This command will start the deployment creation and return a confirmation response while the deployment creation continues.

In [ ]:
ml_client.begin_create_or_update(deployment_keras).result()

### 5.6 Test the endpoint with sample data
Using the `MLClient` created earlier, we will get a handle to the endpoint. The endpoint can be invoked using the `invoke` command with the following parameters:
- `name` - Name of the endpoint
- `input_path` - Path where input data is present
- `deployment_name` - Name of the specific deployment to test in an endpoint

In [ ]:
import time

# Let's sleep for 2 min to ensure all resources are ready. This is only for automation purposes.
time.sleep(120)

#### 5.6.1 Invoke the endpoint

Let's now invoke the endpoint for batch scoring job:

In [ ]:
job = ml_client.batch_endpoints.invoke(
    endpoint_name=endpoint_name,
    deployment_name=deployment_keras.name,
    input=Input(
        path=Path(temp_data_path) / "data",
        type=AssetTypes.URI_FOLDER,
    ),
)

#### 5.6.2 Get the details of the invoked job
Let us get details and logs of the invoked job

In [ ]:
ml_client.jobs.get(job.name)

We can wait for the job to finish using the following code:

In [ ]:
ml_client.jobs.stream(job.name)

#### 5.6.3 Download the results

The deployment creates a child job that executes the scoring. We can get the details of it using the following code:

In [ ]:
scoring_job = list(ml_client.jobs.list(parent_job_name=job.name))[0]

In [ ]:
print("Job name:", scoring_job.name)
print("Job status:", scoring_job.status)
print(
    "Job duration:",
    scoring_job.creation_context.last_modified_at
    - scoring_job.creation_context.created_at,
)

The outputs generated by the deployment job will be placed in an output named `score`:

In [ ]:
ml_client.jobs.download(name=scoring_job.name, download_path=".", output_name="score")

We can read this data using pandas library:

In [ ]:
import pandas as pd

score = pd.read_csv(
    "named-outputs/score/predictions.csv",
    header=None,
    names=["file", "class"],
    sep=" ",
)
score

# 6. Change the default deployment

Now that we know the deployment works, we can switch from one deployment to the new one.

In [ ]:
endpoint = ml_client.batch_endpoints.get(endpoint_name)
endpoint.defaults.deployment_name = deployment_keras.name
ml_client.batch_endpoints.begin_create_or_update(endpoint).result()

We can also delete the old deployment:

In [ ]:
ml_client.batch_deployments.begin_delete(
    endpoint_name=endpoint_name, name=deployment.name
).result()

# 7. Clean up Resources
Delete endpoint

In [ ]:
ml_client.batch_endpoints.begin_delete(name=endpoint_name)

Delete the temp data folder

In [ ]:
import shutil

data_folder = Path(temp_data_path) / "data"
if data_folder.exists():
    shutil.rmtree(data_folder)
    print(f"Deleted {data_folder}")